In [1]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Load trained face recognition model
model = load_model("../models/classification_face_recognition.h5", compile=False, safe_mode=False)

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 160, 160, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_7'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
def get_embedding(img_path):
    img = image.load_img(img_path, target_size=(160, 160))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    embedding = model.predict(img_array)
    return embedding

Found 380638 images belonging to 4000 classes.
Found 8000 images belonging to 4000 classes.


In [ ]:
pairs_file = "../data/verification_pairs_test.txt"
scores, labels = [], []

with open(pairs_file) as f:
    for line in f:
        img1, img2, label = line.strip().split()
        emb1 = get_embedding("../data/" + img1)
        emb2 = get_embedding("../data/" + img2)
        sim = cosine_similarity(emb1, emb2)[0][0]
        scores.append(sim)
        labels.append(int(label))

# ROC/AUC evaluation
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(labels, scores)
print("AUC:", auc)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[0][0]']      

In [ ]:
for t in np.arange(0.3, 0.9, 0.1):
    matches = [s > t for s in scores]
    acc = np.mean([m == bool(l) for m, l in zip(matches, labels)])
    print(f"Threshold {t:.1f} → Accuracy: {acc:.3f}")